<h1>Akmel Syed - Finding Lane Lines</h1><br/>Import all the libraries needed for the project

In [3]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import math
from moviepy.editor import VideoFileClip
from IPython.display import HTML

All of the functions defined in the next cell are the functions provided with the project

In [4]:
def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)
    
def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((*img.shape, 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img
    
def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

The function below uses the above functions to add lines to a video

In [114]:
def process_image(image):
    imshape = image.shape
    #turn the color into grayscale
    gray = grayscale(image)
    
    kernel_size = 5
    #smooth the image to rid it of any excess noise
    blur_gray = gaussian_blur(gray, kernel_size)
    
    #set the upper and lower threshold to make sure only the very solid/thick lines remain
    low_threshold = 50
    high_threshold = 150
    edges = canny(blur_gray, low_threshold, high_threshold)
        
    left = (100,imshape[0])
    right = (imshape[1],imshape[0])
    apex = (imshape[1]/2,imshape[0]/2+50)
    #create the dimensions of the triangle
    vertices = np.array([[left, right, apex]], dtype=np.int32)
    #use that triangle to focus only on the portion directly infront of the car
    masked_edges = region_of_interest(edges, vertices)
    
    rho = 1
    theta = np.pi/180
    threshold = 10
    min_line_length = 50
    max_line_gap = 110
    #using the hough transform functionaltiy, find the lane lines
    lines = hough_lines(masked_edges, rho, theta, threshold, min_line_length, max_line_gap)
    
    #draw the red lines on top of the lane lines to point out where they are
    lines_edges = weighted_img(lines, image)
    return lines_edges

Take in the solidWhiteRight.mp4 and output white.mp4 (i.e. the video which has the lane lines pointed out)

In [115]:
white_output = 'white.mp4'
clip1 = VideoFileClip("solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video white.mp4
[MoviePy] Writing video white.mp4


100%|███████████████████████████████████████████████████████████████████████████████▋| 221/222 [00:03<00:00, 63.51it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: white.mp4 

Wall time: 3.93 s


Take in the solidYellowLeft.mp4 and output yellow.mp4 (i.e. the video which has the lane lines pointed out)

In [116]:
yellow_output = 'yellow.mp4'
clip1 = VideoFileClip("solidYellowLeft.mp4")
yellow_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time yellow_clip.write_videofile(yellow_output, audio=False)

[MoviePy] >>>> Building video yellow.mp4
[MoviePy] Writing video yellow.mp4


100%|███████████████████████████████████████████████████████████████████████████████▉| 681/682 [00:11<00:00, 58.36it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: yellow.mp4 

Wall time: 12.2 s


<h2>Reflection</h2><br/>Throughout this project, I learned several things. I've learned that a lot of computer vision has to do with corners and edges and also that pictures are better dealt with in black and white, rather than in color. My pipeline takes an image, turns it into a grayscale image, thereafter using the canny edge detection to find edges within a certain threshold. After that, creating a triangle, it focuses on a small portion of the picture where only the lane lines lie. Using the hough transformation algorithm, it finds the lane lines and thereafter colors in the lines. Finally returning the altered picture, containing the in focus lane lines drawn.<br/><br/>My pipeline works on the first 2 videos quite well, but breaks when it attempts the challenge. I believe that the shortcoming my pipeline has is that it doesn't deal well with differing lighting/coloring on the road. If you run the challenge, you'll see the when the road gets lighter, the pipeline doesn't work at all, because my pipeline really takes advantage of the contrasts between the very black road and really white lines given in other 2 videos. One of the ways I believe to overcome this problem is to use a method of machine learning where it estimates about where the line would appear, by keeping track of them in the previous frames, and then draw those estimations on the road.